In [1]:
%matplotlib notebook 
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm 
import matplotlib.animation as anim
import sympy as sym

In [15]:
class Particle():
    
    # init
    def __init__(self, r0,v0,a0,t,m,radius,Id):
        
        self.dt  = t[1] - t[0]
        
        self.r = r0
        self.v = v0
        self.a = a0
        self.rVector = np.zeros( (len(t),len(r0)) )
        self.vVector = np.zeros( (len(t),len(v0)) )
        self.aVector = np.zeros( (len(t),len(a0)) )
        self.m = m
        self.radius = radius
        self.Id = Id
        self.Fuerza = self.m*self.a
        self.M = self.m*self.v
        self.Momento = np.zeros( (len(t),len(v0)) )
        self.EnergiaCineticaV = np.zeros((len(t),1))
        self.EnergiaPotencialV = np.zeros((len(t),1))
        self.EnergiaPotencial = 0
        
    def Evolution(self,i):
        
        self.SetPosition(i,self.r)
        self.SetVelocity(i,self.v)
        self.SetMomento(i,self.M)
        
        self.r += self.dt * self.v
        
        self.a = self.Fuerza/self.a
        self.SetEnergiaCinetica(i, self.GetEnergiaCinetica())
        self.SetEnergiaPotencial(i, self.GetEnergiaPotencial())
        
        self.v += self.dt * self.a
        
    #Se define un metodo para restaurar la fuerza    
    def RestartFuerza(self):
        self.Fuerza[:] = 0
        self.a[:] = 0
        self.EnergiaPotencial = 0
        self.Niter = 0
        
        
        
    def CheckWallLimits(self,limits,dim=2):
        
        for i in range(dim):
            if self.r[i] + self.radius > limits[i]:
                self.v[i] = - self.v[i]
            if self.r[i] - self.radius < - limits[i]:
                self.v[i] = - self.v[i]
                


    
    def SetPosition(self,i,r):
        self.rVector[i] = r
        
    def SetVelocity(self,i,v):
        self.vVector[i] = v
        
    def SetMomento(self,i,p):
        self.Momento[i] = p
    
    def SetEnergiaCinetica(self,i,Ek):
        self.EnergiaCineticaV[i] = Ek
        
    def SetEnergiaPotencial(self,i,Ep):
        self.EnergiaPotencialV[i] = Ep
        
    def SetFuerza(self, particula):
        k = 100
        distancia = np.linalg.norm(self.r - particula.GetPosition())
        Compresion = self.radius + particula.GetR() - distancia
        
        if Compresion > 0:
            Fuerza = (k* Cmpresion**3) /d
            self.Fuerza = np.add(self.Fuerza, Fuerza*(self.r - particula.GetPosition()))
            self.EnergiaPotencial += k*Compresion**4/4 
    
           
    def GetPosition(self):
        return self.r
    
    def GetPositionVector(self):
        return self.rVector
    
    def GetRPositionVector(self):
        return self.RrVector 
    
    def GetReduceVelocity(self):
        return self.RvVector
    
    def GetReducePosition(self):
        return self.RrVector

    def GetVelocityVector(self):
        return self.vVector
    
    def GetR(self):
        return self.radius
    
    def GetMomentoVector(self):
        return self.Momento
    
    def GetEnergiaCinetica(self):
        return (1/2)*self.m * np.linalg.norm(self.v)*2
    
    def GetEnergiaPotencial(self):
        return (1/2)*self.EnergiaPotencial /2
    
    def GetFuerza(self):
        return self.Fuerza
        
            
        
        
    
    def ReduceSize(self,factor):
        
        self.RrVector = np.array([self.rVector[0]]) 
        
        
        for i in range(1,len(self.rVector)):
            if i%factor == 0:
                self.RrVector = np.vstack([self.RrVector,self.rVector[i]])
        
        self.RrVector = np.array([self.vVector[0]])
        
        for i in range(1,len(self.vVector)):
            if i%factor == 0:
                self.RvVector = np.vstack([self.RvVector,self.vVector[i]])

In [16]:
dt = 0.01
tmax = 10
t = np.arange(0,tmax+dt,dt)
print(len(t))
1001
def GetParticles(Limit,Dim=2,dt=0.1):
    
    Particles_ = []
    for i in range(3):
        if i == 0:
            
            x0 = np.array([-15.,15.])
            v0 = np.array([20.,0.])
            a0 = np.array([0,-0])
            p = Particle(x0,v0,a0,t,1.,1.0,i)
            Particles_.append(p)
            
        if i == 1:
            
            x0 = np.array([0.,-1.6])
            v0 = np.array([0.,0.])
            a0 = np.array([0,0])
            p = Particle(x0,v0,a0,t,1.,1.0,i)
            Particles_.append(p)
        
        if i == 2:
            
            x0 = np.array([-15.,-15.])
            v0 = np.array([0.,0.])
            a0 = np.array([0,0])
            p = Particle(x0,v0,a0,t,1.,1.0,i)
            Particles_.append(p)
        
    return Particles_

Limits = np.array([20,20])

def RunSimulation(t,NParticles = 3):
    
    Particles = GetParticles(Limits[1],dt=dt)
    
    for it in tqdm(range(len(t))): 
        
        
        for i in range(len(Particles)):
            
            for j in range(len(Particles)):
            
                if i !=j:
                    Particles[i].SetFuerza(Particles[j])
                    
        for i in range(len(Particles)):
                
            Particles[i].CheckWallLimits(Limits)
            Particles[i].Evolution(it)
            Particles[i].RestartFuerza()
        
        
    return Particles


Particles = RunSimulation(t)

def ReduceTime(t,factor):
    
    for p in Particles:
        p.ReduceSize(factor)
        
    Newt = []
    
    for i in range(len(t)):
        if i%factor == 0:
            Newt.append(t[i])
            
    return np.array(Newt)

redt = ReduceTime(t,40)
fig = plt.figure(figsize=(5,5))
ax = fig.add_subplot(1,1,1)


def init():
    ax.set_xlim(-Limits[0],Limits[0])
    ax.set_ylim(-Limits[1],Limits[1])
    
    

def Update(i):
    
    plot = ax.clear()
    init()
    plot = ax.set_title(r'$t=%.2f \ seconds$' %(redt[i]), fontsize=15)
    
    for p in Particles:
        x = p.GetRPositionVector()[i,0]
        y = p.GetRPositionVector()[i,1]
        
        vx = p.GetVelocityVector()[i,0]
        vy = p.GetVelocityVector()[i,1]
        
        circle = plt.Circle( (x,y), p.GetR(), color='k', fill=False)
        plot = ax.add_patch(circle)
        plot = ax.arrow(x,y,vx,vy,color='r',head_width=0.5)
        
    return plot

        
Animation = anim.FuncAnimation(fig,Update,frames=len(redt),init_func=init)

1001


100%|██████████| 1001/1001 [00:01<00:00, 883.51it/s]


AttributeError: 'Particle' object has no attribute 'RvVector'